In [2]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure 
from bokeh.layouts import row, column
from ipywidgets import interact
output_notebook()

Loading BokehJS ...

In [9]:
# Defining parameters
k1 = 0.2  # 1/s   # put this to 0
k2 = 0.6  # 1/s   #lambda # Add T which describes the delay
vmax = 25.0  # m/s
tdes = 1.0  # s
hbuf = 1.06
alpha_p = 0.4  # m/s^2
vp = 15.0  # m/s
tp = 10.0  # s
d = 7.5 # m
ts1 = 20.0  # s
init_sep = (d + tdes * vmax) * hbuf
timelist = []
poslist = []
tmax = 400
num_cars = 200
timesteps = 100
car_pos = np.zeros(timesteps)
init_car_pos = np.linspace(num_cars * init_sep, 0, num_cars) 
car_vel = np.zeros(timesteps)
init_car_vel = np.empty(num_cars)
init_car_vel.fill(vmax)

In [10]:
# Setup plots
# Figure 1
p1 = figure(plot_width=500, plot_height=500, title='Time-Space Diagram without JAD',
           x_axis_label='Time', y_axis_label='Position')
r11 = p1.multi_line(timelist, poslist)

In [11]:
# def RunJAD():
def LeadingCar(t, y):
    x, v = y
    dydt = [v, a]
    return dydt
def FollowingCar(t, y):
    x, v = y
    dydt = [v, -(k2 + k1 * tdes) * v - k1 * x + vfront(t) * k2 + xfront(t) * k1 - k1 * d]  # solve this usinng a dde solver
    return dydt
def SolveCarFollow(a, t1, t2):
    y0 = [init_car_pos[0], init_car_vel[0]]
    sol = solve_ivp(LeadingCar, (t1, t2), y0, method='RK45', t_eval=timepoints)
    global car_pos, car_vel
    car_pos = np.copy(sol.y[0, :])
    car_vel = np.copy(sol.y[1, :])
    np.clip(car_vel, 0, vmax, out=car_vel)
    init_car_pos[0] = car_pos[-1]
    init_car_vel[0] = car_vel[-1]
    timelist.append(sol.t)
    poslist.append(car_pos)
    for i in range(1, num_cars):
        y0 = [init_car_pos[i], init_car_vel[i]]
        global xfront, vfront
        xfront = interp1d(timepoints, car_pos, bounds_error=False, fill_value='extrapolate')
        vfront = interp1d(timepoints, car_vel, bounds_error=False, fill_value='extrapolate')
        sol = solve_ivp(FollowingCar, (t1, t2), y0, method='RK45', t_eval=timepoints)
        car_pos = np.copy(sol.y[0, :])
        car_vel = np.copy(sol.y[1, :])
        np.clip(car_vel, 0, vmax, out=car_vel)
        init_car_pos[i] = car_pos[-1]
        init_car_vel[i] = car_vel[-1]
        if np.mod(i, 5) == 0:
            timelist.append(sol.t)
            poslist.append(car_pos)

In [12]:
# Constant velocity phase 1
a = 0
tl1 = 0
tl2 = ts1
timepoints = np.linspace(tl1, tl2, timesteps)
xfront = interp1d(timepoints, car_pos)
vfront = interp1d(timepoints, car_vel)
SolveCarFollow(a, 0, ts1)

# Constant deceleration phase
a = -alpha_p
tl3 = tl2 + (vp - vmax) / a
timepoints = np.linspace(tl2, tl3, timesteps)
SolveCarFollow(a, tl2, tl3)

# Constant velocity phase 2
a = 0
tl4 = tl3 + tp
timepoints = np.linspace(tl3, tl4, timesteps)
SolveCarFollow(a, tl3, tl4)

# Constant acceleration phase
a = alpha_p
tl5 = tl4 + (vmax - vp) / a
timepoints = np.linspace(tl4, tl5, timesteps)
SolveCarFollow(a, tl4, tl5)

# Constant velocity phase 3
a = 0
tl6 = tmax
timepoints = np.linspace(tl5, tl6, timesteps)
SolveCarFollow(a, tl5, tl6)

In [13]:
r11.data_source.data['x'] = timelist
r11.data_source.data['y'] = poslist
push_notebook(handle=target1)

In [14]:
target1 = show(p1, notebook_handle=True)